# Реализация нахождения уравнений движения с помощью Урванений Татаринова 

##### TODO 
 - Добавить рисунок и описание решаемой здесь задачи
 - Добавить решение с помощью общих теорем динамики
 - Тесты

In [1]:
import sympy as sp
from sympy import Derivative, Symbol, IndexedBase, Eq, Idx, Sum, Function, Matrix
from sympy import diff, symbols, solve, simplify, poly, pprint
from sympy import cos, sin

from abc import ABC, abstractmethod

### Уравнения Татаринова
$$
\frac{d}{dt} \frac{\partial L*}{\partial \omega_{\alpha}} +
\{P_{\alpha}, L*\} = 
\{P_{\alpha}, \sum_{\mu}{\omega_{\mu} P_{\mu}} \} +
\sum_{i}{Q_i \frac{\partial v_i}{ \partial \omega_{\alpha}}}
$$

### Скобка Пуассона
$$
\{f, g\} = \sum_{i=1}^{N} \left( \frac{\partial f}{\partial q_{i}} \frac{\partial g}{\partial p_{i}} - \frac{\partial f}{\partial p_i} \frac{\partial g}{\partial q_i}\right).
$$

### Функции $ P_{\alpha} $
$$
\sum_{\alpha}{P_{\alpha} \omega_{\alpha}} = \sum_i{p_iv_i}
$$

### Функции $v_i$
$$
v_i = \dot{q}
$$

In [2]:
class MechanicalSystem():
    
    def __init__(self):
        p = IndexedBase('p')
        self.p = [p[1], p[2], p[3]]
        self.q = [x, y, alpha]
    
    def poisson_bracket(self, F, G): # надо сделать, чтобы использовалось IndexedBase
        """ To evaluate an unevaluated derivative, use the doit method. """
        res = 0
        for i in range(3):
            res += diff(F, self.q[i])*diff(G, self.p[i])
            res -= diff(F, self.p[i])*diff(G, self.q[i])
        return res
    
    @staticmethod
    def sub_Eq(equation, Eq):
        return equation.subs(Eq.args[0], Eq.args[1])
    
    @classmethod
    def sub_Eqs(cls, equation, Eqs):
        sub_equation = equation
        for Eq in Eqs:
            sub_equation = cls.sub_Eq(sub_equation, Eq)
        return sub_equation 
    
    
    @staticmethod
    def left_part_Eqs(Eqs):
        return [Eq.args[0] for Eq in Eqs]
    
    @staticmethod
    def right_part_Eqs(Eqs):
        return [Eq.args[1] for Eq in Eqs]
    
    def set_connections(self, connections):
        """ Setting up connections """
        self.connections = connections
        
    def sub_connections(self, func):
        """ Substitutes connections in the equation
        Private special case:
            L -> L* 
        """
        sub_dict = {conn.args[0]: conn.args[1] for conn in self.connections}
        return simplify(func.subs(sub_dict))
    
    def sub_connections_to_list(self, equations):
        return [self.sub_connections(equation) for equation in equations]
    
    @staticmethod
    def diff_hack(equation, by):
        """ eq -> eq.subs(Derivative -> temp_variable) -> eq.diff(temp_variable) ->
            -> eq.subs(temp_variable -> Derivative)
        """
        tmp_by = Symbol('tmp_by')
        tmp_eq = equation.subs({by: tmp_by})
        tmp_eq = tmp_eq.diff(tmp_by)
        return tmp_eq.subs({tmp_by: by})
        

In [3]:
class TatarinovSystem(MechanicalSystem):
    
    def __init__(self, N=3):
        super().__init__()
        self.N = N
        self.tatarinov_equations = [None]*N
        
    def set_q(self):
        pass
    
    def set_p(self):
        pass

    def set_omega_equations(self, omegas, equations):
        omega_equations = [Eq(omega, equation) for omega, equation in zip(omegas, equations)]
        self.omega_equations = omega_equations
        
    def set_v_equations(self, vs, equations): # vs - не лучшее название
        v_equations = [Eq(v, equation) for v, equation in zip(vs, equations)]
        self.v_equations = v_equations
    
    def set_v(self):
        v = IndexedBase('v')
        k= symbols('k', cls=Idx)
        Eq(v[k], diff(self.q[k]))
        
    def set_P(self):
        k, mu = symbols('k, mu', cls=Idx)
        left = Sum(P[k]*om[k], (k, 1, self.N)).doit()
        right = Sum(p[i]*v[i], (k, 1, self.N))
        Eq(left, right)
    
    def set_L(self):
        pass
    
    def set_F(self, F):
        self.F = F
    
    def create_r(self):
        r = {}
        r['s'] = x*e['x'] + y*e['y']
        r['p'] = r['s'] + xi*e['xi'] + eta*e['eta']
        self.r = r
        return self.r

    def set_Qs(self, Qs):
        self.Qs = Qs
    
    def create_P(self):
        equations_for_P = Eq(
            Sum(P[k]*omega[k], (k, 1, S.N)),
            Sum(p[k]*v[k], (k, 1, S.N)))
        print(equations_for_P)
        # subs omega_i and v_i
        sub_equations_for_P = TatarinovSystem.sub_Eqs(
            equations_for_P.doit(), # important
            self.omega_equations + self.v_equations) # for Ex: omega[1] -> nu1), v[1] -> x', ....
        # subs_connections
        sub_conn_equations_for_P = self.sub_connections(sub_equations_for_P)
        print(sub_conn_equations_for_P)
        # to equate the coefficients
        left_coeffs = poly(sub_conn_equations_for_P.args[0], TatarinovSystem.right_part_Eqs(self.omega_equations)).coeffs()
        right_coeffs = poly(sub_conn_equations_for_P.args[1], TatarinovSystem.right_part_Eqs(self.omega_equations)).coeffs()
        final_equations_for_P = [Eq(left, right) for left, right in zip(left_coeffs, right_coeffs)]
        self.Ps = final_equations_for_P
        return final_equations_for_P
    
    def create_Q(self, F, r_p):
        self.Q = [F.dot(r_p.diff(x)) for x in self.q]
    
    def Q_dw_by_dv(self, i):
        Q_dw_by_dv = lambda i,a: self.Q[i]*Derivative(self.right_part_Eqs(S.sub_connections_to_list(S.v_equations))[i],
                                                      self.right_part_Eqs(self.omega_equations)[a]).doit()
        Q_dw_by_dv_sum = lambda a: Q_dw_by_dv(0,a) + Q_dw_by_dv(1,a) + Q_dw_by_dv(2,a)
        return Q_dw_by_dv_sum(i)
    
    def tatarinov_equation__depricated(self, i):
        left, right = 0, 0;
        left += diff(diff(L, omega[k]), t)
        left += poisson_bracket(P[k], L)
        right += poisson_bracket(P[k], Sum(omega[k]*P[k], (k, 1, self.N)))
        right += Sum(Q[l]*diff(v[l], omega[k]), (l, 1, self.N))
        Eq(left, right)
        
    def create_bracket_sum(self):
        """ not good """
        bracket_sum = nu1*(p[1]*cos(alpha) + p[2]*sin(alpha)) + nu2*(-p[1]*sin(alpha) + p[2]*cos(alpha)) + nu3*p[3]
        return bracket_sum
    
    def create_fs(self, fs):
        """ not good """
        return fs.subs({
            nu1: Function('nu1')(t),
            nu2: Function('nu2')(t),
            nu3: Function('nu3')(t)
        })

    def tatarinov_equation(self, i):
        left, right = 0,0
        left = S.sub_connections(L).args[1]
        left = left.diff(self.right_part_Eqs(self.omega_equations)[i])
        left += self.poisson_bracket(self.Ps[i].args[1], self.sub_connections(L).args[1])
        left = self.create_fs(left).diff(t)
        
        right += self.poisson_bracket(self.Ps[i].args[1], self.create_bracket_sum())
        right += simplify(self.Q_dw_by_dv(i))
        right = self.create_fs(right)
        
        self.tatarinov_equations[i] = simplify(Eq(left, right))
        
        return self.tatarinov_equations[i]

    def display_tatarinov_equations(self):
        for eq in self.tatarinov_equations:
            display(eq)

In [4]:
L, m, x, y, t, alpha = symbols('L, m, x, y, t, alpha')
nu1, nu2, nu3 = symbols('nu1, nu2, nu3')
W, T = symbols('W, T')
s = symbols('s')
P = IndexedBase('P')
p = IndexedBase('p')
omega = IndexedBase('omega')
v = IndexedBase('v')
Q = IndexedBase('Q')
k = symbols('k', cls=Idx)
J = IndexedBase('J')

xi, eta = symbols('xi, eta')
e = {}
e['x'] = Matrix([1,0])
e['y'] = Matrix([0,1])
e['xi'] = Matrix([cos(alpha), sin(alpha)])
e['eta'] = Matrix([-sin(alpha), cos(alpha)])

### Определим Лагранжиан

In [5]:
L = Eq(L, m*(Derivative(x,t)**2 + Derivative(y,t)**2)/2 + J[s]*(Derivative(alpha, t)**2)/2)
L

Eq(L, m*(Derivative(x, t)**2 + Derivative(y, t)**2)/2 + Derivative(alpha, t)**2*J[s]/2)

### Определим связи 

In [6]:
conn1 = Eq(Derivative(x,t), cos(alpha)*nu1 - sin(alpha)*nu2)
conn2 = Eq(Derivative(y,t), sin(alpha)*nu1 + cos(alpha)*nu2)
conn3 = Eq(Derivative(alpha,t), nu3)

### Создадим объект для механической системы с уравнениями Татаринова 

In [7]:
S = TatarinovSystem()

### Заполним что нужно (связи, уравнения для $\omega_i$, $v_i$, $F$)

In [8]:
S.set_connections([conn1, conn2, conn3])
S.connections

[Eq(Derivative(x, t), nu1*cos(alpha) - nu2*sin(alpha)),
 Eq(Derivative(y, t), nu1*sin(alpha) + nu2*cos(alpha)),
 Eq(Derivative(alpha, t), nu3)]

In [9]:
S.set_omega_equations(omegas=[omega[1], omega[2], omega[3]],
                      equations=[nu1, nu2, nu3])
S.omega_equations

[Eq(omega[1], nu1), Eq(omega[2], nu2), Eq(omega[3], nu3)]

In [10]:
S.set_v_equations(vs=[v[1], v[2], v[3]],
                  equations=[Derivative(x,t), Derivative(y,t), Derivative(alpha,t)])
S.v_equations

[Eq(v[1], Derivative(x, t)),
 Eq(v[2], Derivative(y, t)),
 Eq(v[3], Derivative(alpha, t))]

In [11]:
F = W*e['xi'] + T*e['eta']
S.set_F(F)

### Создадим $P_{\alpha}$

In [12]:
Ps = S.create_P()
Ps

Eq(Sum(P[k]*omega[k], (k, 1, 3)), Sum(p[k]*v[k], (k, 1, 3)))
Eq(nu1*P[1] + nu2*P[2] + nu3*P[3], nu3*p[3] + (nu1*sin(alpha) + nu2*cos(alpha))*p[2] + (nu1*cos(alpha) - nu2*sin(alpha))*p[1])


[Eq(P[1], sin(alpha)*p[2] + cos(alpha)*p[1]),
 Eq(P[2], -sin(alpha)*p[1] + cos(alpha)*p[2]),
 Eq(P[3], p[3])]

### Получение $Q_i$

In [13]:
S.create_Q(S.F, S.create_r()['p'])
S.Q

[-T*sin(alpha) + W*cos(alpha),
 T*cos(alpha) + W*sin(alpha),
 (-T*sin(alpha) + W*cos(alpha))*(-eta*cos(alpha) - xi*sin(alpha)) + (T*cos(alpha) + W*sin(alpha))*(-eta*sin(alpha) + xi*cos(alpha))]

### Получаем уравнения

In [14]:
for i in range(3):
    S.tatarinov_equation(i)

In [15]:
S.display_tatarinov_equations()

Eq(m*Derivative(nu1(t), t), W - (sin(alpha)*p[1] - cos(alpha)*p[2])*nu3(t))

Eq(m*Derivative(nu2(t), t), T - (sin(alpha)*p[2] + cos(alpha)*p[1])*nu3(t))

Eq(Derivative(nu3(t), t)*J[s], (sin(alpha)*p[1] - cos(alpha)*p[2])*nu1(t) + (sin(alpha)*p[2] + cos(alpha)*p[1])*nu2(t))

In [27]:
fps = lambda i: S.sub_connections(S.diff_hack(L.args[1], S.right_part_Eqs(S.v_equations)[i]))
ps = [fps(x) for x in range(3)]
ps

[m*(nu1*cos(alpha) - nu2*sin(alpha)),
 m*(nu1*sin(alpha) + nu2*cos(alpha)),
 nu3*J[s]]

In [32]:
for x in range(3):
    display(simplify(S.tatarinov_equations[x]))

Eq(m*Derivative(nu1(t), t), W - (sin(alpha)*p[1] - cos(alpha)*p[2])*nu3(t))

Eq(m*Derivative(nu2(t), t), T - (sin(alpha)*p[2] + cos(alpha)*p[1])*nu3(t))

Eq(Derivative(nu3(t), t)*J[s], (sin(alpha)*p[1] - cos(alpha)*p[2])*nu1(t) + (sin(alpha)*p[2] + cos(alpha)*p[1])*nu2(t))

### Тесты

In [69]:
S.poisson_bracket(Ps[0].args[1], S.sub_connections(L).args[1])

0

In [70]:
S.poisson_bracket(Ps[1].args[1], S.sub_connections(L).args[1])

0

In [71]:
S.poisson_bracket(Ps[2].args[1], S.sub_connections(L).args[1])

0

In [57]:
tmp = nu1*(p[1]*cos(alpha) + p[2]*sin(alpha)) + nu2*(-p[1]*sin(alpha) + p[2]*cos(alpha)) + nu3*p[3]

In [22]:
S.poisson_bracket(Ps[0].args[1], tmp)

(-sin(alpha)*p[1] + cos(alpha)*p[2])*Derivative(alpha, t)

In [23]:
S.poisson_bracket(Ps[1].args[1], tmp)

(-sin(alpha)*p[2] - cos(alpha)*p[1])*Derivative(alpha, t)

In [24]:
S.poisson_bracket(Ps[2].args[1], tmp)

-nu1*(-sin(alpha)*p[1] + cos(alpha)*p[2]) - nu2*(-sin(alpha)*p[2] - cos(alpha)*p[1])

In [25]:
# def create_Q(F, r_p):
#     return [F.dot(r_p.diff(x)) for x in self.q]

### Всё что связано с векторами

In [19]:
F = W*e['xi'] + T*e['eta']
F

Matrix([
[-T*sin(alpha) + W*cos(alpha)],
[ T*cos(alpha) + W*sin(alpha)]])

In [20]:
r = {}
r['s'] = x*e['x'] + y*e['y']
r['p'] = r['s'] + xi*e['xi'] + eta*e['eta']
r['p']

Matrix([
[-eta*sin(alpha) + x + xi*cos(alpha)],
[ eta*cos(alpha) + xi*sin(alpha) + y]])

In [21]:
r['p'].diff(x)

Matrix([
[1],
[0]])

In [22]:
r['p'].diff(y)

Matrix([
[0],
[1]])

In [23]:
r['p'].diff(alpha)

Matrix([
[-eta*cos(alpha) - xi*sin(alpha)],
[-eta*sin(alpha) + xi*cos(alpha)]])

### Получение $Q_i\frac{\partial v_i}{\partial w_{\alpha}}$

In [35]:
l = lambda i,a: S.Q[i]*Derivative(MechanicalSystem.right_part_Eqs(S.sub_connections_to_list(S.v_equations))[i],
                                  MechanicalSystem.right_part_Eqs(S.omega_equations)[a]).doit()
ll = lambda a: l(0,a) + l(1,a) + l(2,a)

In [39]:
# S.sub_connections_to_list(S.v_equations)

In [40]:
ll(0)

(-T*sin(alpha) + W*cos(alpha))*cos(alpha) + (T*cos(alpha) + W*sin(alpha))*sin(alpha)

### Уравнения Татаринова
$$
\frac{d}{dt} \frac{\partial L*}{\partial \omega_{\alpha}} +
\{P_{\alpha}, L*\} = 
\{P_{\alpha}, \sum_{\mu}{\omega_{\mu} P_{\mu}} \} +
\sum_{i}{Q_i \frac{\partial v_i}{ \partial \omega_{\alpha}}}
$$

In [132]:
# [x.diff() for x in S.right_part_Eqs(S.connections)

In [133]:
S.right_part_Eqs(S.v_equations)

[Derivative(x, t), Derivative(y, t), Derivative(alpha, t)]

In [134]:
S.right_part_Eqs(S.omega_equations)

[nu1, nu2, nu3]

### Получаем уравнения 

In [135]:
S.omega_equations

[Eq(omega[1], nu1), Eq(omega[2], nu2), Eq(omega[3], nu3)]

In [136]:
MechanicalSystem.right_part_Eqs(S.omega_equations)

[nu1, nu2, nu3]

In [137]:
L.args[1]

m*(Derivative(x, t)**2 + Derivative(y, t)**2)/2 + Derivative(alpha, t)**2*J[s]/2

##### Разберемся с  $$ \frac{\partial L}{\partial \omega_{\alpha}} $$

In [138]:
# for x in S.right_part_Eqs(S.v_equations):
#     left+=
# S.diff_hack(L.args[1], x) for x in ]

In [139]:
# S.diff_hack(L.args[1], Derivative(y,t))

In [140]:
# S.diff_hack(L.args[1], Derivative(alpha,t))

In [141]:
# xx = symbols('xx')
# diff(Derivative(x, t).subs(Derivative(x, t), xx), xx)

In [41]:
def create_fs(fs):
    return fs.subs(
    {
        nu1: Function('nu1')(t),
        nu2: Function('nu2')(t),
        nu3: Function('nu3')(t)
    }
)


In [143]:
S.sub_connections(L).args[1]

m*(nu1**2 + nu2**2)/2 + nu3**2*J[s]/2

In [144]:
l1 = lambda x: S.sub_connections(L).args[1].diff(x)
    
# create_fs(left).diff(t)

### Уравнения

In [213]:
left, right = 0,0

In [214]:
i = 0

### Уравнения Татаринова
$$
\frac{d}{dt} \frac{\partial L*}{\partial \omega_{\alpha}} +
\{P_{\alpha}, L*\} = 
\{P_{\alpha}, \sum_{\mu}{\omega_{\mu} P_{\mu}} \} +
\sum_{i}{Q_i \frac{\partial v_i}{ \partial \omega_{\alpha}}}
$$

In [215]:
# left += create_fs(S.sub_connections(L).args[1])
# left

# left = left.diff(create_fs(MechanicalSystem.right_part_Eqs(S.omega_equations)[i]))
# left
L

Eq(L, m*(Derivative(x, t)**2 + Derivative(y, t)**2)/2 + Derivative(alpha, t)**2*J[s]/2)

In [216]:
left = S.sub_connections(L)
left = left.args[1]
left

m*(nu1**2 + nu2**2)/2 + nu3**2*J[s]/2

In [217]:
left = left.diff(nu1)
left

m*nu1

In [218]:
S.v_equations

[Eq(v[1], Derivative(x, t)),
 Eq(v[2], Derivative(y, t)),
 Eq(v[3], Derivative(alpha, t))]

In [219]:
fps = lambda i: S.sub_connections(S.diff_hack(L.args[1], S.right_part_Eqs(S.v_equations)[i]))
ps = [fps(x) for x in range(3)]
ps

[m*(nu1*cos(alpha) - nu2*sin(alpha)),
 m*(nu1*sin(alpha) + nu2*cos(alpha)),
 nu3*J[s]]

In [220]:
Ps[0].args[1]

sin(alpha)*p[2] + cos(alpha)*p[1]

In [221]:
S.sub_connections(L).args[1]

m*(nu1**2 + nu2**2)/2 + nu3**2*J[s]/2

In [222]:
S.poisson_bracket(Ps[0].args[1], S.sub_connections(L).args[1])

0

In [25]:
left += S.poisson_bracket(Ps[0].args[1], S.sub_connections(L).args[1])
left = create_fs(left).diff(t)

NameError: name 'left' is not defined

In [26]:
bracket_sum = nu1*(p[1]*cos(alpha) + p[2]*sin(alpha)) + nu2*(-p[1]*sin(alpha) + p[2]*cos(alpha)) + nu3*p[3]
bracket_sum

nu1*(sin(alpha)*p[2] + cos(alpha)*p[1]) + nu2*(-sin(alpha)*p[1] + cos(alpha)*p[2]) + nu3*p[3]

In [225]:
S.poisson_bracket(Ps[0].args[1], bracket_sum)

nu3*(-sin(alpha)*p[1] + cos(alpha)*p[2])

In [226]:
right += S.poisson_bracket(Ps[0].args[1], bracket_sum)
right

nu3*(-sin(alpha)*p[1] + cos(alpha)*p[2])

In [227]:
simplify(ll(0))

W

In [228]:
right += simplify(ll(0))
right

W + nu3*(-sin(alpha)*p[1] + cos(alpha)*p[2])

In [229]:
right = create_fs(right)

In [230]:
Eq(left, right)

Eq(m*Derivative(nu1(t), t), W + (-sin(alpha)*p[1] + cos(alpha)*p[2])*nu3(t))

In [231]:
RES = Eq(left, right).subs({p[1]: ps[0], p[2]: ps[1], p[3]: ps[2]})

In [232]:
RES

Eq(m*Derivative(nu1(t), t), W + (m*(nu1*sin(alpha) + nu2*cos(alpha))*cos(alpha) - m*(nu1*cos(alpha) - nu2*sin(alpha))*sin(alpha))*nu3(t))

In [233]:
simplify(RES)

Eq(W + m*nu2*nu3(t), m*Derivative(nu1(t), t))

In [52]:
S.Q[2].simplify()

T*xi - W*eta

In [43]:
bracket_sum = nu1*(p[1]*cos(alpha) + p[2]*sin(alpha)) + nu2*(-p[1]*sin(alpha) + p[2]*cos(alpha)) + nu3*p[3]

In [48]:
for i in range(3):
    left, right = 0,0
    left = S.sub_connections(L)
    left = left.args[1]
    left = left.diff(S.right_part_Eqs(S.omega_equations)[i])
    left += S.poisson_bracket(Ps[i].args[1], S.sub_connections(L).args[1])
    left = create_fs(left).diff(t)
    right += S.poisson_bracket(Ps[i].args[1], bracket_sum)
    right += simplify(ll(i))
    right = create_fs(right)
    
    display(simplify(Eq(left, right)))

Eq(m*Derivative(nu1(t), t), W - (sin(alpha)*p[1] - cos(alpha)*p[2])*nu3(t))

Eq(m*Derivative(nu2(t), t), T - (sin(alpha)*p[2] + cos(alpha)*p[1])*nu3(t))

Eq(Derivative(nu3(t), t)*J[s], (sin(alpha)*p[1] - cos(alpha)*p[2])*nu1(t) + (sin(alpha)*p[2] + cos(alpha)*p[1])*nu2(t))

In [87]:
# Interface

MechanicalSystem.set_q()
MechanicalSystem.set_p()
# ...

AttributeError: type object 'MechanicalSystem' has no attribute 'set_q'

In [ ]:
v[1]